In [2]:
import requests as re
from bs4 import BeautifulSoup

__Get HTML__

In [3]:
url = 'https://www.dol.gov/agencies/whd/minimum-wage/state'
response = re.get(url)

In [45]:
html = response.text
soup = BeautifulSoup(html, "html.parser")
cat = soup.find('div',{"id": "states"})
state = cat.find_all('div')